# Prediksi dan Deteksi Anomali Konsumsi Listrik Rumah Tangga

In [ ]:

from sklearn.tree import DecisionTreeRegressor

# Decision Tree Regressor untuk prediksi konsumsi listrik
reg_model = DecisionTreeRegressor(random_state=42)
reg_model.fit(X_train_scaled, y_train)

y_pred = reg_model.predict(X_test_scaled)

# Evaluasi
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error:", rmse)
plt.figure(figsize=(10, 4))
plt.plot(y_test.values, label='Actual')
plt.plot(y_pred, label='Predicted', linestyle='dashed')
plt.title("Perbandingan Konsumsi Listrik Aktual vs Prediksi")
plt.legend()
plt.show()


## Simulasi Dataset Penggunaan Listrik Sehari-hari (Sintetik)

In [ ]:

# Dataset buatan: penggunaan listrik per hari selama 60 hari
np.random.seed(42)
days = pd.date_range(start="2024-01-01", periods=60, freq='D')
daily_usage = np.random.normal(loc=15, scale=5, size=60).clip(min=5)  # kWh
peak_usage = daily_usage * np.random.uniform(0.2, 0.6, size=60)
num_devices = np.random.randint(3, 15, size=60)

# Simulasi anomali (misalnya korsleting listrik)
is_short_circuit = [1 if i in [10, 35, 48] else 0 for i in range(60)]  # hanya beberapa hari

df = pd.DataFrame({
    'date': days,
    'day_usage_kwh': daily_usage,
    'peak_hour_usage': peak_usage,
    'num_devices_on': num_devices,
    'is_short_circuit': is_short_circuit
})
df.set_index('date', inplace=True)
df.head()


## Visualisasi Awal Penggunaan Listrik

In [ ]:

plt.figure(figsize=(12,5))
plt.plot(df.index, df['day_usage_kwh'], label='Pemakaian Harian (kWh)')
plt.scatter(df[df['is_short_circuit']==1].index, df[df['is_short_circuit']==1]['day_usage_kwh'], color='red', label='Konsleting', zorder=5)
plt.title("Penggunaan Listrik Harian dan Konsleting")
plt.xlabel("Tanggal")
plt.ylabel("kWh")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


## Prediksi Konsumsi Listrik (Regresi)

In [ ]:

from sklearn.tree import DecisionTreeRegressor

# Decision Tree Regressor untuk prediksi konsumsi listrik
reg_model = DecisionTreeRegressor(random_state=42)
reg_model.fit(X_train_scaled, y_train)

y_pred = reg_model.predict(X_test_scaled)

# Evaluasi
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error:", rmse)
plt.figure(figsize=(10, 4))
plt.plot(y_test.values, label='Actual')
plt.plot(y_pred, label='Predicted', linestyle='dashed')
plt.title("Perbandingan Konsumsi Listrik Aktual vs Prediksi")
plt.legend()
plt.show()


## Deteksi Anomali Penggunaan Listrik

In [ ]:

features = df[['day_usage_kwh', 'peak_hour_usage', 'num_devices_on']]
features_scaled = scaler.fit_transform(features)

anomaly_model = IsolationForest(contamination=0.1, random_state=42)
df['anomaly'] = anomaly_model.fit_predict(features_scaled)
df['anomaly'] = df['anomaly'].map({1: 0, -1: 1})  # 1 = anomali

# Tampilkan data anomali
df[df['anomaly'] == 1]


In [ ]:

plt.figure(figsize=(12,5))
plt.plot(df.index, df['day_usage_kwh'], label='Pemakaian Harian (kWh)')
plt.scatter(df[df['anomaly']==1].index, df[df['anomaly']==1]['day_usage_kwh'], color='orange', label='Anomali', zorder=5)
plt.scatter(df[df['is_short_circuit']==1].index, df[df['is_short_circuit']==1]['day_usage_kwh'], color='red', label='Konsleting', marker='x', zorder=5)
plt.title("Deteksi Anomali dan Konsleting Listrik")
plt.xlabel("Tanggal")
plt.ylabel("kWh")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


## Klasifikasi Konsleting Menggunakan KNN

In [ ]:

from sklearn.tree import DecisionTreeClassifier

# Fitur dan label untuk klasifikasi konsleting
X_class = df[['day_usage_kwh', 'peak_hour_usage', 'num_devices_on']]
y_class = df['is_short_circuit']

# Split data
Xc_train, Xc_test, yc_train, yc_test = train_test_split(X_class, y_class, test_size=0.2, random_state=42)

# Scaling
Xc_train_scaled = scaler.fit_transform(Xc_train)
Xc_test_scaled = scaler.transform(Xc_test)

# Model Decision Tree untuk klasifikasi
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(Xc_train_scaled, yc_train)
yc_pred = dt_model.predict(Xc_test_scaled)

# Evaluasi Decision Tree
print("Akurasi Decision Tree:", accuracy_score(yc_test, yc_pred))
print("Laporan Klasifikasi:
", classification_report(yc_test, yc_pred))
